In [1]:
import pandas as pd
import psycopg2

# Connection settings
conn = psycopg2.connect(
    dbname="healthguardian",
    user="postgres",
    password="Akash@2001",
    host="localhost",
    port="5432"
)

In [3]:
df_patients = pd.read_sql("SELECT * FROM patients;", conn)
df_patients.head()

C:\Users\akash\AppData\Local\Temp\ipykernel_30928\2990427546.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_patients = pd.read_sql("SELECT * FROM patients;", conn)


,id,birthdate,deathdate,gender,race,ethnicity
0,74fb7318-9e69-35ea-266c-3bee58c2d31a,2010-03-11,None,M,white,hispanic
1,c044b671-1ff3-483e-40de-63d5266cfe6a,2010-05-11,None,M,white,nonhispanic
2,a3d550b9-d110-fa1a-6871-1ba8c3dd542b,1959-07-23,None,M,white,nonhispanic
3,04a1cbab-08c8-fb9c-8285-13b8bc847905,2020-04-18,None,F,white,hispanic
4,645b3172-3c6b-8d3a-d092-b50478f7f8a8,1974-01-12,None,F,white,nonhispanic


In [4]:
df_encounters = pd.read_sql("SELECT * FROM encounters;", conn)
df_encounters.head()

C:\Users\akash\AppData\Local\Temp\ipykernel_30928\3015909148.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_encounters = pd.read_sql("SELECT * FROM encounters;", conn)


,id,patient_id,start,stop,encounter_class,description
0,9a290e20-8f61-e7b4-1ce2-b620bded1fc0,74fb7318-9e69-35ea-266c-3bee58c2d31a,2015-02-19 05:32:42,2015-02-19 05:47:42,wellness,Well child visit (procedure)
1,3ebbf6a4-5d78-7fed-5ede-513a5f8c0f33,74fb7318-9e69-35ea-266c-3bee58c2d31a,2016-02-25 05:32:42,2016-02-25 05:47:42,wellness,Well child visit (procedure)
2,ebca7554-e7fc-4ef6-fd2d-f9969792f8e5,74fb7318-9e69-35ea-266c-3bee58c2d31a,2017-03-02 05:32:42,2017-03-02 05:47:42,wellness,Well child visit (procedure)
3,34878f8b-11f2-1b23-b7a4-f40a586ea78b,74fb7318-9e69-35ea-266c-3bee58c2d31a,2017-03-09 05:32:42,2017-03-09 09:12:55,ambulatory,Encounter for check up (procedure)
4,ab2c4781-6b14-1c4c-b6c6-87e02a3d3833,c044b671-1ff3-483e-40de-63d5266cfe6a,2013-10-15 19:14:30,2013-10-15 19:29:30,wellness,Well child visit (procedure)


In [5]:
df_conditions = pd.read_sql("SELECT * FROM conditions;", conn)
df_conditions.head()

C:\Users\akash\AppData\Local\Temp\ipykernel_30928\1926216512.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_conditions = pd.read_sql("SELECT * FROM conditions;", conn)


,patient_id,start,stop,description


In [6]:
print("Patients nulls:")
print(df_patients.isnull().sum())

print("\nEncounters nulls:")
print(df_encounters.isnull().sum())

print("\nConditions nulls:")
print(df_conditions.isnull().sum())


Patients nulls:
id             0
birthdate      0
deathdate    100
gender         0
race           0
ethnicity      0
dtype: int64

Encounters nulls:
id                 0
patient_id         0
start              0
stop               0
encounter_class    0
description        0
dtype: int64

Conditions nulls:
patient_id     0
start          0
stop           0
description    0
dtype: int64


In [7]:
# Create base dataframe with patient info
df_features = df_patients.copy()

# Compute age
df_features['birthdate'] = pd.to_datetime(df_features['birthdate'])
df_features['age'] = (pd.Timestamp('today') - df_features['birthdate']).dt.days // 365

# Keep only needed columns
df_features = df_features[['id', 'gender', 'race', 'ethnicity', 'age']]
df_features.head()



,id,gender,race,ethnicity,age
0,74fb7318-9e69-35ea-266c-3bee58c2d31a,M,white,hispanic,15
1,c044b671-1ff3-483e-40de-63d5266cfe6a,M,white,nonhispanic,15
2,a3d550b9-d110-fa1a-6871-1ba8c3dd542b,M,white,nonhispanic,66
3,04a1cbab-08c8-fb9c-8285-13b8bc847905,F,white,hispanic,5
4,645b3172-3c6b-8d3a-d092-b50478f7f8a8,F,white,nonhispanic,51


In [8]:
# Convert start to datetime
df_encounters['start'] = pd.to_datetime(df_encounters['start'])

# Filter encounters in past 12 months
cutoff_date = pd.Timestamp('today') - pd.DateOffset(months=12)
recent_encounters = df_encounters[df_encounters['start'] >= cutoff_date]

# Count
encounter_counts = recent_encounters.groupby('patient_id').size().reset_index(name='encounters_last_12m')
encounter_counts.head()


,patient_id,encounters_last_12m
0,000ea483-2b67-863e-74b9-cc2b5ad2be71,9
1,04a1cbab-08c8-fb9c-8285-13b8bc847905,2
2,080f5aac-fdeb-912c-d379-cb7e1b94995d,6
3,09d7f977-c502-eb7f-2d21-247f0ee3c4e3,1
4,0aaecf6a-1fd0-d2b3-4f98-16f56060ae3b,4


In [9]:
# Merge counts
df_features = df_features.merge(encounter_counts, how='left', left_on='id', right_on='patient_id')

# Fill NaN with 0
df_features['encounters_last_12m'] = df_features['encounters_last_12m'].fillna(0).astype(int)

# Drop redundant column
df_features = df_features.drop(columns=['patient_id'])

df_features.head()


,id,gender,race,ethnicity,age,encounters_last_12m
0,74fb7318-9e69-35ea-266c-3bee58c2d31a,M,white,hispanic,15,1
1,c044b671-1ff3-483e-40de-63d5266cfe6a,M,white,nonhispanic,15,18
2,a3d550b9-d110-fa1a-6871-1ba8c3dd542b,M,white,nonhispanic,66,6
3,04a1cbab-08c8-fb9c-8285-13b8bc847905,F,white,hispanic,5,2
4,645b3172-3c6b-8d3a-d092-b50478f7f8a8,F,white,nonhispanic,51,18


In [10]:
# Define target: High utilization if >1 encounter
df_features['high_utilization'] = (df_features['encounters_last_12m'] > 1).astype(int)
df_features.head()


,id,gender,race,ethnicity,age,encounters_last_12m,high_utilization
0,74fb7318-9e69-35ea-266c-3bee58c2d31a,M,white,hispanic,15,1,0
1,c044b671-1ff3-483e-40de-63d5266cfe6a,M,white,nonhispanic,15,18,1
2,a3d550b9-d110-fa1a-6871-1ba8c3dd542b,M,white,nonhispanic,66,6,1
3,04a1cbab-08c8-fb9c-8285-13b8bc847905,F,white,hispanic,5,2,1
4,645b3172-3c6b-8d3a-d092-b50478f7f8a8,F,white,nonhispanic,51,18,1


In [11]:
from sklearn.model_selection import train_test_split

# Select features and target
X = df_features[['age', 'encounters_last_12m']]
y = df_features['high_utilization']

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

X_train.shape, X_test.shape


((87, 2), (22, 2))

In [12]:
from sklearn.linear_model import LogisticRegression

# Create model
model = LogisticRegression()

# Train
model.fit(X_train, y_train)


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [13]:
from sklearn.metrics import classification_report, confusion_matrix

# Predict
y_pred = model.predict(X_test)

# Evaluate
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Confusion Matrix:
[[ 7  0]
 [ 0 15]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         7
           1       1.00      1.00      1.00        15

    accuracy                           1.00        22
   macro avg       1.00      1.00      1.00        22
weighted avg       1.00      1.00      1.00        22



In [14]:
# Example: 65-year-old patient with 2 encounters
example = pd.DataFrame({
    'age': [65],
    'encounters_last_12m': [2]
})

prediction = model.predict(example)
print("Predicted High Utilization (1=yes):", prediction[0])


Predicted High Utilization (1=yes): 1


In [16]:
from sklearn.ensemble import RandomForestClassifier

# Create the model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train
rf_model.fit(X_train, y_train)


,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [17]:
# Predict
rf_preds = rf_model.predict(X_test)

# Evaluate
from sklearn.metrics import classification_report

print("Random Forest Classification Report:")
print(classification_report(y_test, rf_preds))


Random Forest Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         7
           1       1.00      1.00      1.00        15

    accuracy                           1.00        22
   macro avg       1.00      1.00      1.00        22
weighted avg       1.00      1.00      1.00        22



In [18]:
pip install xgboost


Note: you may need to restart the kernel to use updated packages.


In [19]:
import xgboost as xgb

# Create DMatrix (XGBoost format)
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Parameters
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'seed': 42
}

# Train
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round=100
)


In [20]:
# Predict probabilities
xgb_preds_prob = xgb_model.predict(dtest)

# Convert to 0/1
xgb_preds = (xgb_preds_prob > 0.5).astype(int)


In [21]:
print("XGBoost Classification Report:")
print(classification_report(y_test, xgb_preds))


XGBoost Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         7
           1       1.00      1.00      1.00        15

    accuracy                           1.00        22
   macro avg       1.00      1.00      1.00        22
weighted avg       1.00      1.00      1.00        22



In [22]:
from sklearn.metrics import accuracy_score

print("Logistic Regression Accuracy:", accuracy_score(y_test, model.predict(X_test)))
print("Random Forest Accuracy:", accuracy_score(y_test, rf_preds))
print("XGBoost Accuracy:", accuracy_score(y_test, xgb_preds))


Logistic Regression Accuracy: 1.0
Random Forest Accuracy: 1.0
XGBoost Accuracy: 1.0


In [29]:
# Predict high utilization for all patients using Random Forest
df_features['predicted_high_utilization'] = rf_model.predict(
    df_features[['age', 'encounters_last_12m']]
)

df_features.head()


,id,gender,race,ethnicity,age,encounters_last_12m,high_utilization,predicted_high_utilization,xgb_predicted_high_utilization
0,74fb7318-9e69-35ea-266c-3bee58c2d31a,M,white,hispanic,15,1,0,0,0
1,c044b671-1ff3-483e-40de-63d5266cfe6a,M,white,nonhispanic,15,18,1,1,1
2,a3d550b9-d110-fa1a-6871-1ba8c3dd542b,M,white,nonhispanic,66,6,1,1,1
3,04a1cbab-08c8-fb9c-8285-13b8bc847905,F,white,hispanic,5,2,1,1,1
4,645b3172-3c6b-8d3a-d092-b50478f7f8a8,F,white,nonhispanic,51,18,1,1,1


In [32]:
import xgboost as xgb

# Create DMatrix for all patients
d_all = xgb.DMatrix(df_features[['age', 'encounters_last_12m']])

# Predict probabilities
xgb_preds_prob = xgb_model.predict(d_all)

# Convert probabilities to 0/1 predictions
df_features['xgb_predicted_high_utilization'] = (xgb_preds_prob > 0.5).astype(int)



df_features.to_csv(
    r"C:\Users\akash\OneDrive\Desktop\HealthGuardian360\data\patient_predictions_with_rf_xgb.csv",
    index=False
)

print("✅ CSV file saved.")


✅ CSV file saved.
